In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "./input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

sample_submission.csv
test.csv
train.csv



In [2]:
train = pd.read_csv('./input/train.csv')
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [1]:
%%bash
wget http://nlp.stanford.edu/data/glove.6B.zip
unzip glove.6B.zip

--2017-10-27 18:10:07--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... failed: Name or service not known.
wget: unable to resolve host address ‘nlp.stanford.edu’
unzip:  cannot find or open glove.6B.zip, glove.6B.zip.zip or glove.6B.zip.ZIP.


In [17]:
import nltk as nl
train['tokens'] = [nl.word_tokenize(sentences) for sentences in train.text]
words = []
for item in train.tokens:
    words.extend(item)

stemmer = nl.stem.lancaster.LancasterStemmer()
words = [stemmer.stem(word) for word in words]


filtered_words = [word for word in words if word not in nl.corpus.stopwords.words('english')]



import gensim
# let X be a list of tokenized texts (i.e. list of lists of tokens)
model = gensim.models.Word2Vec(filtered_words, size=100)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

In [24]:
w2v['h']

array([-0.48447788,  0.37772679,  0.1721192 , -0.25281611, -0.08350382,
       -0.26236266, -0.35221589, -0.11461841,  0.01211224,  0.16695961,
        0.36798939, -0.24858071, -0.30548489, -0.1226057 , -0.01497782,
       -0.42093369,  0.19981885, -0.60778433, -0.25410685,  0.21789254,
        0.27064028, -0.01399691,  0.06578006,  0.11984596,  0.27820253,
        0.48097903, -0.21241115,  0.2023008 , -0.15852831, -0.06108602,
       -0.28014383,  0.23098896,  0.25369716, -0.28983158,  0.22965187,
       -0.05795765,  0.34624857, -0.20967057, -0.14075291, -0.29322013,
       -0.46175703,  0.04668606, -0.4968957 ,  0.06816328,  0.2058243 ,
       -0.06381508, -0.2770367 ,  0.23208925,  0.2076458 ,  0.04258641,
        0.0881149 , -0.16849409,  0.20704637,  0.21805969, -0.17064475,
        0.37618664,  0.16684383, -0.15153997, -0.37234405, -0.21195473,
        0.25754294,  0.15656258,  0.52061534, -0.65176928, -0.25596833,
        0.55746633,  0.20862029,  0.06659539,  0.36561462,  0.27

In [94]:
training = []
for index,item in train.iterrows():
    vec = np.zeros(100)
    token_words = [stemmer.stem(word) for word in item['tokens']]
    token_words = [word for word in token_words if word not in nl.corpus.stopwords.words('english')]
    for w in token_words:
        if w in w2v:
            vec += w2v[w]
    norm = np.linalg.norm(vec)
    if norm != 0:
        vec /= np.linalg.norm(vec)
    
    
    training.append(vec)

In [6]:
training_new = np.array(training)

In [7]:
from numpy import array

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(training_new[:,1])

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

train_y = onehot_encoded

In [8]:
train_x = list(training_new[:,0])


In [9]:
import tensorflow as tf
import tflearn
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
 
# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=10, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 24479  | total loss: 1.07462 | time: 8.087s
| Adam | epoch: 010 | loss: 1.07462 - acc: 0.4479 -- iter: 19576/19579
Training Step: 24480  | total loss: 1.07736 | time: 8.090s
| Adam | epoch: 010 | loss: 1.07736 - acc: 0.4406 -- iter: 19579/19579
--
INFO:tensorflow:/Users/dmurugan/mess/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [11]:
test = pd.read_csv('./input/test.csv')
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [12]:
test['tokens'] = [nl.word_tokenize(sentences) for sentences in test.text]


In [13]:
testing = []
for index,item in test.iterrows():
    vec = np.zeros(100)
    token_words = [stemmer.stem(word) for word in item['tokens']]
    token_words = [word for word in token_words if word not in nl.corpus.stopwords.words('english')]
    for w in token_words:
        if w in w2v:
            vec += w2v[w]
    norm = np.linalg.norm(vec)
    if norm != 0:
        vec /= np.linalg.norm(vec)
    
    
    testing.append(vec)

In [14]:
testing = list(np.array(testing))
predicted = model.predict(X=testing)
result_val = pd.DataFrame(predicted)
result_val.columns = ["EAP","HPL","MWS"]
result = pd.DataFrame(columns=['id'])
result['id'] = test['id']
result['EAP'] = result_val['EAP']
result['HPL'] = result_val['HPL']
result['MWS'] = result_val['MWS']
result.head()

,id,EAP,HPL,MWS
0,id02310,0.391960,0.295293,0.312747
1,id24541,0.399334,0.274379,0.326287
2,id00134,0.388437,0.299305,0.312257
3,id27757,0.395750,0.284641,0.319608
4,id04081,0.387966,0.306310,0.305724


In [15]:
result.to_csv("my_submission_v2.csv", encoding='utf-8',index=False)